### <b>Image augmentation for INPUT: <b style="color:red">128x128</b></b>

In [1]:
import os
import cv2
import numpy as np
import shutil

In [2]:
class Augmentation:
    def __init__(self, random_bound=0.5):
        self.random_bound = random_bound
    
    def augment_image(self, image, rotate=True):
        # Randomly flip the image horizontally (left-right)
        if np.random.rand() > self.random_bound:
            image = cv2.flip(image, 1)

        # Fixed rotations (90°, 45°, 180°)
        if rotate:
            # Choose a random fixed angle from the list
            angle = np.random.choice([90, 180, 270])
            height, width = image.shape[:2]
            center = (width // 2, height // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
            image = cv2.warpAffine(image, rotation_matrix, (width, height))

        # Random brightness adjustment
        factor = np.random.uniform(1.0, 1.05)
        image = cv2.convertScaleAbs(image, alpha=factor, beta=0)
        return image
    
    def augment_image_file(self, path_to_input_image, path_to_output_image, rotate=True):
        image = cv2.imread(path_to_input_image)
        image = self.augment_image(image)
        cv2.imwrite(path_to_output_image, image)

#### HELPING ####

In [ ]:
n_images = 180
augmenter = Augmentation()
path = "./final-dataset/imbalanced_flags/S"
category = "S"
files = os.listdir(path)
last_id = -1
for file in files:
    print(file)
    id = int(file.split("_")[1].split(".")[0])
    if id > last_id:
        last_id = id
for i in range(n_images):
    selected_id = np.random.randint(0, last_id-1) + 1
    if selected_id < 10:
        selected_id = f"0{selected_id}"
    last_id = last_id + 1
    path_to_selected_image = f"{path}/{category}_{selected_id}.jpg"
    path_to_generate_image = f"{path}/{category}_{last_id}.jpg"
    print(f"Path to selected image: {path_to_selected_image}")
    print(f"Path to generate image: {path_to_generate_image}\n")
    augmenter.augment_image_file(
        path_to_selected_image,
        path_to_generate_image,
        True
    )

R_01.jpg
R_02.jpg
R_03.jpg
R_04.jpg
R_05.jpg
R_06.jpg
R_07.jpg
R_08.jpg
R_09.jpg
R_10.jpg
R_100.jpg
R_101.jpg
R_102.jpg
R_103.jpg
R_104.jpg
R_105.jpg
R_106.jpg
R_107.jpg
R_108.jpg
R_109.jpg
R_11.jpg
R_110.jpg
R_111.jpg
R_112.jpg
R_113.jpg
R_114.jpg
R_115.jpg
R_116.jpg
R_117.jpg
R_118.jpg
R_119.jpg
R_12.jpg
R_120.jpg
R_121.jpg
R_122.jpg
R_123.jpg
R_124.jpg
R_125.jpg
R_126.jpg
R_127.jpg
R_128.jpg
R_129.jpg
R_13.jpg
R_130.jpg
R_131.jpg
R_132.jpg
R_133.jpg
R_134.jpg
R_135.jpg
R_136.jpg
R_137.jpg
R_138.jpg
R_139.jpg
R_14.jpg
R_140.jpg
R_141.jpg
R_142.jpg
R_143.jpg
R_144.jpg
R_145.jpg
R_146.jpg
R_147.jpg
R_148.jpg
R_149.jpg
R_15.jpg
R_150.jpg
R_151.jpg
R_152.jpg
R_153.jpg
R_154.jpg
R_155.jpg
R_156.jpg
R_157.jpg
R_158.jpg
R_159.jpg
R_16.jpg
R_160.jpg
R_161.jpg
R_162.jpg
R_163.jpg
R_164.jpg
R_165.jpg
R_166.jpg
R_167.jpg
R_168.jpg
R_169.jpg
R_17.jpg
R_170.jpg
R_171.jpg
R_172.jpg
R_173.jpg
R_174.jpg
R_175.jpg
R_176.jpg
R_177.jpg
R_178.jpg
R_179.jpg
R_18.jpg
R_180.jpg
R_181.jpg
R_182.jpg
R_183.jp

AttributeError: 'NoneType' object has no attribute 'shape'

#### -------------------- ####

In [245]:
class AugmentationBalancer:
    def __init__(self, random_bound=0.5):
        self.augmentation = Augmentation(random_bound=random_bound)
    
    def fit(self, path_to_folder, delta=5, debug=False):
        self.path_to_folder=path_to_folder        
        directories = os.listdir(path_to_folder)
        real_directories = {}
        self.to_generate = {}
        for directory in directories:
            if os.path.isdir(f"{path_to_folder}/{directory}"):
                count = len(os.listdir(f"{path_to_folder}/{directory}"))
                real_directories[directory]=count
        max_id = max(real_directories)
        self.total_classes = [key for key in real_directories]
        for directory in real_directories:
            diff = real_directories[max_id] - real_directories[directory]
            if(diff>delta):
                self.to_generate[directory]=diff
        if debug:
            print(self.to_generate)
        
    def balance(self, path_to_folder, debug=False):
        for category in self.total_classes:
            source_folder = f"{self.path_to_folder}/{category}"
            destination_folder =f"{path_to_folder}/{category}"
            if os.path.exists(destination_folder) and os.path.isdir(destination_folder):
                shutil.rmtree(destination_folder)
            shutil.copytree(source_folder, destination_folder)
        if debug:
            print(self.to_generate)
        for category in self.to_generate:
            last_id = -1
            for file in os.listdir(f"{path_to_folder}/{category}"):
                id = int(file.split("_")[1].split(".")[0])
                if id > last_id:
                    last_id = id
            for i in range(self.to_generate[category]):
                selected_id = np.random.randint(0, last_id-1) + 1
                if selected_id < 10:
                    selected_id = f"0{selected_id}"
                last_id = last_id + 1
                path_to_selected_image = f"{path_to_folder}/{category}/{category}_{selected_id}.jpg"
                path_to_generate_image = f"{path_to_folder}/{category}/{category}_{last_id}.jpg"
                if debug:
                    print(f"Path to selected image: {path_to_selected_image}")
                    print(f"Path to generate image: {path_to_generate_image}\n")
                self.augmentation.augment_image_file(
                    path_to_selected_image,
                    path_to_generate_image
                )
    
aug_balancer = AugmentationBalancer()
aug_balancer.fit("./dataset/training/imbalanced_flags", debug=True)
aug_balancer.balance("./dataset/training/balanced_flags/", debug=True)

{'China': 20, 'Germany': 7, 'India': 57}
{'China': 20, 'Germany': 7, 'India': 57}
Path to selected image: ./dataset/training/balanced_flags//China/China_72.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_80.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_03.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_81.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_73.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_82.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_75.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_83.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_02.jpg
Path to generate image: ./dataset/training/balanced_flags//China/China_84.jpg

Path to selected image: ./dataset/training/balanced_flags//China/China_05.jpg
Path to generate image: ./dataset/training/balanced_fla